In [ ]:
rm(list = ls())
setwd("/home/creambbq/facu/TEA/minimum_wages")
library("haven");library("dplyr")
data <- read_dta('datos.dta')


Creo las variables de interés y las de separación: 

In [14]:
data <- data %>% group_by(state) %>% 
  mutate(fte_before = empft + nmgrs + emppt*0.5,
         fte_after = empft2 + nmgrs2 + emppt2*0.5, 
         wage_seg = case_when(wage_st == 4.25 ~ "low", 
                              wage_st >= 5 ~ "hight",
                              between(wage_st,4.26, 4.99 ) ~ "midrange"))%>% 
  ungroup()

Esta primera parte replica las 3 primeras filas y dos primeras columnas (notar que falta la tercera columna que es la resta de las otras dos pero no sé como sabar el desvio estándar ://) 

In [3]:
first <- data %>% 
  group_by(state) %>% 
  summarise(mean_before = mean(fte_before, na.rm = TRUE),
            mean_after = mean(fte_after, na.rm = TRUE),
            var_before = var(fte_before, na.rm = TRUE),
            var_after = var(fte_after, na.rm = TRUE),
            n_before = sum(!is.na(fte_before)),
            n_after = sum(!is.na(fte_after))) %>%
  mutate(se_mean_before = sqrt(var_before/n_before), 
         se_mean_after = sqrt(var_after/n_after), 
         state = dplyr::recode(state, '0' = "PA", '1' = "NJ"), 
         change_mean_fte = mean_after - mean_before) %>%
  select(state,
         mean_before,
         se_mean_before,
         mean_after,
         se_mean_after, 
         change_mean_fte)

first


state,mean_before,se_mean_before,mean_after,se_mean_after,change_mean_fte
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PA,23.33117,1.3511489,21.16558,0.9432212,-2.1655844
NJ,20.43941,0.5082607,21.02743,0.5203094,0.5880214


Esta es la fila inmediatamente inferior, con el panel balanceado. Mismo problema de Desvío estándar: 

In [5]:
second <- data %>%
  filter(complete.cases(fte_before, fte_after)) %>% #Esto balancea el panel
  group_by(state) %>% 
  summarise(mean_before_balanced = mean(fte_before),
            mean_after_balanced = mean(fte_after))  %>% 
  mutate(change_mean_fte_balanced = mean_after_balanced - mean_before_balanced,
         state = dplyr::recode(state, '0' = "PA", '1' = "NJ")) %>% 
  select(state, change_mean_fte_balanced)

second

state,change_mean_fte_balanced
<chr>,<dbl>
PA,-2.2833333
NJ,0.4666667


###  De acá en adelante es el conjunto de columnas "Stores in New Jersey"

primeras tres filas sin el desvío estándar de la última

In [8]:
test <- data %>% 
  filter(state == 1) %>% 
  filter(is.na(wage_seg) == FALSE) %>% 
  group_by(wage_seg) %>% 
  summarise(mean_before = mean(fte_before, na.rm=TRUE),
            mean_after = mean(fte_after, na.rm=TRUE),
            var_before = var(fte_before, na.rm=TRUE),
            var_after = var(fte_after, na.rm=TRUE),
            n_before = sum(!is.na(fte_before)),
            n_after = sum(!is.na(fte_after))) %>%
  mutate(se_mean_before = sqrt(var_before/n_before), 
         se_mean_after = sqrt(var_after/n_after), 
         change_mean_fte = mean_after - mean_before) %>% 
  select(wage_seg, 
         mean_before,
         se_mean_before,
         mean_after,
         se_mean_after, 
         change_mean_fte) %>% 
  ungroup()

test

wage_seg,mean_before,se_mean_before,mean_after,se_mean_after,change_mean_fte
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
hight,22.25000,1.1357994,20.21377,1.0251042,-2.036232
low,19.55670,0.7748557,20.87755,1.0059015,1.320850
midrange,20.15882,0.7817561,21.49342,0.7477344,1.334598


Cuarta fila, pero sin desvío estándar (ah lo aclaraba siempre)

In [16]:
test2 <- data %>% 
  filter(complete.cases(fte_before, fte_after)) %>% 
  filter(state == 1) %>%
  filter(is.na(wage_seg) == FALSE) %>%
  group_by(wage_seg) %>% 
  summarise(mean_before_balanced = mean(fte_before, na.rm = TRUE),
            mean_after_balanced = mean(fte_after, na.rm = TRUE)) %>% 
  mutate(change_mean_fte_balanced = mean_after_balanced - mean_before_balanced) %>% 
  select(wage_seg, change_mean_fte_balanced) %>% 
  ungroup()

test2

wage_seg,change_mean_fte_balanced
<chr>,<dbl>
hight,-2.1567164
low,1.2047872
midrange,0.7098485
